In [ ]:
import os
import random
import itertools
from PIL import Image
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


In [ ]:

# Paths from the original notebook
base_dir = 'C:/Users/ARA/Desktop/ara330/media/image_data'
processed_dir = 'C:/Users/ARA/Desktop/ara330/media/processed_image_data(updated)'


In [ ]:

# Create the processed image directory structure
def create_dir_structure(base_dir, processed_dir):
    for root, dirs, files in os.walk(base_dir):
        for dir_name in dirs:
            dir_path = os.path.join(processed_dir, os.path.relpath(os.path.join(root, dir_name), base_dir))
            os.makedirs(dir_path, exist_ok=True)

create_dir_structure(base_dir, processed_dir)


In [ ]:
# Image preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Function to denormalize the image before saving (for visualization)
def denormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)  # Reverse the normalization
    return tensor

In [ ]:

# Function to load, preprocess, and save images
def load_preprocess_and_save_image(image_path, save_path):
    image = Image.open(image_path).convert('RGB')
    tensor = transform(image)
    denorm_tensor = denormalize(tensor.clone(), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    denorm_image = transforms.ToPILImage()(denorm_tensor)
    denorm_image.save(save_path)

# Process and save images
def process_images(base_dir, processed_dir):
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                image_path = os.path.join(root, file)
                save_path = os.path.join(processed_dir, os.path.relpath(image_path, base_dir))
                load_preprocess_and_save_image(image_path, save_path)

process_images(base_dir, processed_dir)


In [ ]:
# Function to load images from directory
def load_images_from_dir(directory):
    images = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                images.append(os.path.join(root, file))
    return images

In [ ]:

# Function to create and downsample pairs within two categories
def create_and_downsample_pairs(category1, category2, target_size):
    pairs = list(itertools.product(category1, category2))
    if len(pairs) > target_size:
        pairs = random.sample(pairs, target_size)
    return pairs


In [ ]:
# Load and downsample pairs
# Male Informal
male_informal_tops = load_images_from_dir(os.path.join(processed_dir, 'male/informal/tops'))
male_informal_bottoms = load_images_from_dir(os.path.join(processed_dir, 'male/informal/bottoms'))
male_informal_pairs = create_and_downsample_pairs(male_informal_tops, male_informal_bottoms, target_size=191*243)

# Male Formal
male_formal_tops = load_images_from_dir(os.path.join(processed_dir, 'male/formal/tops'))
male_formal_bottoms = load_images_from_dir(os.path.join(processed_dir, 'male/formal/bottoms'))
male_formal_pairs = create_and_downsample_pairs(male_formal_tops, male_formal_bottoms, target_size=207*207)

# Female Formal
female_formal_tops = load_images_from_dir(os.path.join(processed_dir, 'female/formal/tops'))
female_formal_bottoms = load_images_from_dir(os.path.join(processed_dir, 'female/formal/bottoms'))
female_formal_pairs = create_and_downsample_pairs(female_formal_tops, female_formal_bottoms, target_size=35*41)

# Female Informal
female_informal_tops = load_images_from_dir(os.path.join(processed_dir, 'female/informal/tops'))
female_informal_bottoms = load_images_from_dir(os.path.join(processed_dir, 'female/informal/bottoms'))
female_informal_pairs = create_and_downsample_pairs(female_informal_tops, female_informal_bottoms, target_size=1451*1451)

# Combine all pairs
all_pairs = male_informal_pairs + male_formal_pairs + female_formal_pairs + female_informal_pairs

# Print the number of pairs created after downsampling
print(f"Number of pairs created for male informal: {len(male_informal_pairs)}")
print(f"Number of pairs created for male formal: {len(male_formal_pairs)}")
print(f"Number of pairs created for female formal: {len(female_formal_pairs)}")
print(f"Number of pairs created for female informal: {len(female_informal_pairs)}")



In [ ]:
# Custom dataset class for pairs
class FashionPairDataset(Dataset):
    def __init__(self, pairs, transform=None):
        self.pairs = pairs
        self.transform = transform
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        top_image_path, bottom_image_path = self.pairs[idx]
        top_image = Image.open(top_image_path).convert('RGB')
        bottom_image = Image.open(bottom_image_path).convert('RGB')
        
        if self.transform:
            top_image = self.transform(top_image)
            bottom_image = self.transform(bottom_image)
        
        return top_image, bottom_image


In [ ]:
# Create datasets and data loaders
train_dataset = FashionPairDataset(all_pairs, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Use smaller batch size to reduce memory usage


In [ ]:
# Model setup
# Define the feature extractor
feature_extractor = models.resnet50(pretrained=True)
for param in feature_extractor.parameters():
    param.requires_grad = False  # Freeze feature extractor
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])  # Remove the final FC layer

# Define the classifier
classifier = nn.Linear(2048 * 2, 1)  # 2048 is the output size of resnet50's penultimate layer

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor.to(device)
classifier.to(device)


In [ ]:

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.0005)  # Lower learning rate for fine-tuning


In [ ]:

# Training loop
num_epochs = 5  # Fewer epochs to reduce runtime
for epoch in range(num_epochs):
    feature_extractor.eval()  # Set feature extractor to evaluation mode
    classifier.train()       # Set classifier to training mode
    running_loss = 0.0
    for i, (top_images, bottom_images) in enumerate(train_loader):
        top_images = top_images.to(device)
        bottom_images = bottom_images.to(device)
        
        # Extract features
        with torch.no_grad():
            top_features = feature_extractor(top_images)
            top_features = top_features.view(top_features.size(0), -1)  # Flatten
            bottom_features = feature_extractor(bottom_images)
            bottom_features = bottom_features.view(bottom_features.size(0), -1)  # Flatten
        
        # Combine features
        combined_features = torch.cat((top_features, bottom_features), dim=1)  # Concatenate features
        
        # Pass through classifier
        outputs = classifier(combined_features)
        
        # Dummy labels for now (replace with actual labels if available)
        labels = torch.ones(outputs.size(0), 1).to(device)  # Example: all ones
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    average_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

print("Training complete.")

In [ ]:
def extract_features(model, image_paths, device):
    model.eval()  # Set the model to evaluation mode
    features = []
    
    with torch.no_grad():
        for image_path in image_paths:
            image = Image.open(image_path).convert('RGB')
            image = image_transforms(image).unsqueeze(0).to(device)
            feature = model.resnet(image).squeeze().cpu().numpy()
            features.append((image_path, feature))
    
    return features


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_items(selected_item_feature, all_items_features, top_n=5):
    similarities = []
    
    for image_path, feature in all_items_features:
        sim = cosine_similarity([selected_item_feature], [feature])[0][0]
        similarities.append((image_path, sim))
    
    # Sort items by similarity score in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities[:top_n]


In [ ]:
def recommend_bottoms_for_top(model, top_image_path, bottoms_features, device, top_n=5):
    # Extract feature for the selected top
    top_image = Image.open(top_image_path).convert('RGB')
    top_image = image_transforms(top_image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        top_feature = model.resnet(top_image).squeeze().cpu().numpy()
    
    # Recommend bottoms based on similarity
    recommended_bottoms = recommend_items(top_feature, bottoms_features, top_n)
    
    return recommended_bottoms


In [ ]:
# Extract features for all tops and bottoms
top_features = extract_features(model, all_top_image_paths, device)
bottom_features = extract_features(model, all_bottom_image_paths, device)

# Select a top and get recommendations for bottoms
selected_top = 'C:/Users/ARA/Desktop/ara330/media/processed_image_data(updated)/female/formal/tops/18763.png'
recommended_bottoms = recommend_bottoms_for_top(model, selected_top, bottom_features, device, top_n=5)

# Display recommended bottoms
for bottom_path, similarity in recommended_bottoms:
    print(f"Recommended Bottom: {bottom_path}, Similarity: {similarity:.4f}")
